In [252]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import holidays
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from prophet import Prophet


from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

pd.set_option('display.max_rows', None)

In [253]:
def load_bike_data():
  '''Returns DataFrame with cleaned bike usage data'''
  # Load Data
  trips_filenames = [filename for filename in os.listdir() if "trips_data" in filename]
  bike_data = pd.DataFrame()
  for file in trips_filenames:
    temp_df = pd.read_csv(file)
    bike_data = pd.concat([bike_data,temp_df])

  bike_data["Start Time"] = pd.DatetimeIndex(bike_data["Start Time"].str[:-6]).tz_localize('EST')
  bike_data = bike_data.sort_values(by='Start Time').copy()

  # turn time columns into index
  bike_data.index = bike_data["Start Time"]
  bike_data = bike_data.drop(columns=["Start Time"])

  return bike_data


In [254]:
def load_weather_data():
   '''Returns DataFrame with weather data'''
   weather_data = pd.read_csv("weather_data.csv")

    # filter columns
   weather_data = weather_data[["Date/Time","Temp (°C)","Wind Spd (km/h)","Visibility (km)","Weather"]]

   # add datetime as index
   weather_data.index = pd.DatetimeIndex(weather_data["Date/Time"].str[:-6]).tz_localize('EST')
   weather_data = weather_data.sort_index()
   weather_data = weather_data.drop(columns=["Date/Time"])

   return weather_data

In [255]:
def drop_features(data,keep_columns=None,remove_columns=None):
  '''Returns dataframe with irrelevant columns filtered out'''
  if(remove_columns==None):
    return data[keep_columns]
  if(keep_columns==None):
    return data.drop(columns=remove_columns)

def reduce_temporal_precision(data,time_interval="H"):
  '''Returns dataframe with a more condensed time discritization'''
  data["num_trips"] = np.ones(len(data))
  data = data.groupby(data['trip_start_time'].dt.floor(time_interval)).agg({'num_trips':'sum'})
  return data


In [256]:
def add_temporal_features(data):
  '''Returns dataframe with new temporal features hand-crafted for prediction purposes'''
  # add the years since Jan 1,2017 (years can be decimal)
  basedate = pd.Timestamp('2017-01-01',tz="EST")
  data["Years Since Start"] = ((data.index.to_series() - basedate).dt.days)/(365*5)

  # add the day of the year (normalized to 0-1)
  data["Day of Year"] = (data.index.to_series().dt.dayofyear)/365

  # add hour of day (normalized to 0-1)
  data["Hour of Day"] = (data.index.to_series().dt.hour)/24

  # day of week (normalized to 0-1)
  data["Day of Week"] = (data.index.to_series().dt.dayofweek)/7

  # is holiday
  holidays_list = holidays.Canada(years=[2017,2018,2019,2020,2021,2022])
  data["Is Holiday"] = data.index.to_series().dt.date.apply(lambda x: x in holidays_list) 

  # is weekend
  data["Is Weekend"] = data.index.to_series().dt.dayofweek>=5

  return data

In [257]:
def add_weather_features(bike_data, weather_data):
  '''Returns dataframe with new weather features hand-crafted for prediction purposes'''
  # add weather columns
  filt = weather_data["Weather"].isna()
  weather_data["Weather"][filt] = "Clear"
  weather_data["Weather Clear"] = weather_data["Weather"].str.contains("Clear")
  weather_data["Fog"] = weather_data["Weather"].str.contains("Fog")
  weather_data["Rain"] = weather_data["Weather"].str.contains("Rain")
  weather_data["Snow"] = weather_data["Weather"].str.contains("Snow")
  weather_data = weather_data.drop(columns=["Weather"])

  # normalize numeric columns
  weather_data["Temp (°C)"] = (weather_data["Temp (°C)"]-weather_data["Temp (°C)"].mean())/weather_data["Temp (°C)"].std()
  weather_data["Wind Spd (km/h)"] = (weather_data["Wind Spd (km/h)"]-weather_data["Wind Spd (km/h)"].mean())/weather_data["Wind Spd (km/h)"].std()
  weather_data["Visibility (km)"] = (weather_data["Visibility (km)"]-weather_data["Visibility (km)"].mean())/weather_data["Visibility (km)"].std()

  # clean missing values (CHANGE this to take the values of the nearest neighbors)
  print("Weather Temperature Missing",sum(weather_data["Temp (°C)"].isna()),"values")
  weather_data["Temp (°C)"][weather_data["Temp (°C)"].isna()] = weather_data["Temp (°C)"].mean()

  print("Weather Wind Speed Columns Missing",sum(weather_data["Wind Spd (km/h)"].isna()),"values")
  weather_data["Wind Spd (km/h)"][weather_data["Wind Spd (km/h)"].isna()] = weather_data["Wind Spd (km/h)"].mean()

  print("Weather Visibility Missing",sum(weather_data["Visibility (km)"].isna()),"values")
  weather_data["Visibility (km)"][weather_data["Visibility (km)"].isna()] = weather_data["Visibility (km)"].mean()

  print("Weather Time/Date Missing",sum(weather_data.index.to_series().isna()),"values")
  weather_data["Visibility (km)"][weather_data["Visibility (km)"].isna()] = weather_data["Visibility (km)"].mean()

  # merge data
  bike_data = bike_data.merge(weather_data, left_index=True,right_index=True,indicator=True)
  #print(bike_data.head(5))
  bike_data = bike_data.drop(columns=["_merge"])



  return bike_data



In [258]:
def remove_yearly_trend(bike_data):
  '''Attempts to remove the year-on-year growth trend to make prediction easier'''
  '''Not statistically valid. Contains data leakage since trend removal occurs before train-test split, and hence its calibrated
     with test-data'''
  temp_data = bike_data.copy()
  temp_data["num_trips"] = temp_data["num_trips"]*(1-temp_data["Years Since Start"]*0.6)
  plot_data = temp_data.groupby(temp_data.index.to_series().dt.round("30D")).agg({'num_trips':'sum'})
  plt.plot(temp_data)
  plt.show()

  bike_data["num_trips"] = temp_data["num_trips"]

  return bike_data

In [259]:
def convert_continuous_to_discrete(data):
  '''Converts continous time features (ex. Day of week ∈[0,7]) into binary classifications (ex. IsMonday∈{0,1}, IsTuesday∈{0,1}) to help linear
      regression better fit non-linear time trends'''
  data = data.copy()
  time = data.index.to_series()
  data = pd.concat([data,pd.get_dummies(time.dt.month.astype("string"), prefix='Month')], axis=1)
  data = pd.concat([data,pd.get_dummies(time.dt.hour.astype("string"), prefix='Hour')], axis=1)
  data = pd.concat([data,pd.get_dummies(time.dt.dayofweek.astype("string"), prefix='Weekday')], axis=1)

  data = data.drop(columns=["Day of Year","Day of Week","Hour of Day"])

  # Add missing categories in case not encountered
  for i in range(0,13):
    col = "Month_"+str(i)
    if(col not in data.columns):
      data[col] = np.zeros(len(data))
  for i in range(0,25):
    col = "Hour_"+str(i)
    if(col not in data.columns):
      data[col] = np.zeros(len(data))
  for i in range(0,7):
    col = "Weekday_"+str(i)
    if(col not in data.columns):
      data[col] = np.zeros(len(data))

  data = data.sort_index(axis=1)
  #print(data.head(5))

  return data.copy()

In [260]:
#sns.boxplot(x=X_data["Snow"], y=Y_data,showfliers=False)
#plt.show()
#plt.scatter(X_data["Visibility (km)"], Y_data,s=3)

In [261]:
def get_historical_data(date_range,year_range,X_train,Y_train,X_test):
  '''Returns filtered train data, to only include dates at a similar time of year to valid/test range'''
  '''Reduces the requirement for model to understand seasonal trends'''
  '''For example if predicting in Sepetember 2022, return train data with only Sep 2021, Sep 2020, Sep 2019 etc...'''
  days = int((date_range-14)/2)

  filt = np.zeros(len(X_train))
  # For loop runs back from 0 to specified number of years before present
  for years in range(0,10):
      if(years>year_range):
        continue
      # lower and upper specify the valid date range for each year
      lower = X_test.index.to_series().min() - pd.DateOffset(years=years) - pd.DateOffset(days=days)
      upper = X_test.index.to_series().max() - pd.DateOffset(years=years) + pd.DateOffset(days=days)
      #print("Upper",upper,"\nLower",lower)
      temp_filt = (X_train.index.to_series()>=lower) & (X_train.index.to_series()<=upper)
      filt = filt | temp_filt

  X_new = X_train[filt].copy()
  Y_new = Y_train[filt].copy()

  #print("Filtered Historical Data",X_new.index.to_series(),Y_new.index.to_series())
  return X_new, Y_new 


In [262]:
def load_train_data(weather_info=False,remove_yearly_trend=False):
  '''The master function that runs the entire pre-processing pipeline'''
  '''Calls all the othe preprocessing functions'''
  # load bicycle usage and weather information
  bike_data = load_bike_data()
  bike_data = drop_features(bike_data,keep_columns = ["num_trips"], remove_columns=None)
  print("\n\nBike Data:",bike_data.tail(5))
  weather_data = load_weather_data()
  print("\n\nWeather Data:",weather_data.tail(5))

  # add temporal features
  bike_data = add_temporal_features(bike_data)

  # choose whether to add weather features and/or remove the yearly growth trend
  if(weather_info):
    bike_data = add_weather_features(bike_data,weather_data)
  if(remove_yearly_trend):
    bike_data = remove_yearly_trend(bike_data)

  bike_data = bike_data.sort_index()
  Y_data = bike_data["num_trips"].copy()
  X_data = bike_data.drop(columns=["num_trips"])

  return X_data,Y_data


In [263]:
def train_model(X,Y,model,test=False):
  '''Trains the specified model and produces results with time-series cross validation split'''
  '''If test=False, it will show evaluation results on valid data. Else, it tests performs evaluation on test data'''
  rmse = []
  MAE = []
  MAPE = []
  r2 = []

  # here we specifiy the size of the test and valid portions, (1 week each, so both in total get 2 weeks * 7 days/week * 24 hours/day datapoints)
  # to cover the whole year we need 365/(2*7) = 26 splits
  tss = TimeSeriesSplit(test_size = 2*7*24,n_splits=26)
  ind = 0
  Y_test_stacked = pd.Series()
  Y_pred_stacked = pd.Series()

  # The cross validation looop
  for train_index, test_index in tss.split(X):
      ind+=1
      # Instead of 26 splits I resampled 8 evenly spaced splits to save compute time.
      if((ind)%3!=0):
        continue

      # Select to evaluate on test or valid data.
      # Our test/valid split is 2 weeks long. First week is valid, second week is test.
      if(test==True):
        test_index = test_index[0:len(test_index)//2]
      else:
        test_index = test_index[len(test_index)//2:]

      X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

      print("\nTrain Range:",X_train.index.min(),"  ",X_train.index.max())
      print("Test Range:",X_test.index.min(),"  ",X_test.index.max())


      Y_pred = None
      if(model==None):
          continue

      if(model=="Random Forest"):
          rf = RandomForestRegressor(random_state=0,max_depth=10)
          rf.fit(X_train,Y_train)
          Y_pred = rf.predict(X_test)
      
      if(model=="Linear Regression"):
          X_train_temp = X_train; Y_train_temp = Y_train;
          X_train_temp, Y_train_temp = get_historical_data(60,100, X_train, Y_train, X_test)
          X_train_temp = convert_continuous_to_discrete(X_train_temp)
          lr = LinearRegression().fit(X_train_temp, Y_train_temp)
          X_test_temp = convert_continuous_to_discrete(X_test)
          #print(X_train_temp.head(5))
          #print(X_test_temp.head(5))
          Y_pred = lr.predict(X_test_temp)
      
      if(model=="Facebook Prophet"):
          fp = Prophet()

          # format data for facebook prophet
          X_train["ds"] = X_train.index.to_series().dt.tz_convert(None).copy()
          X_train = X_train.reset_index(drop=True)[["ds"]]
          X_train["y"] = Y_train.reset_index(drop=True)
          X_test["ds"] = X_test.index.to_series().dt.tz_convert(None).copy()
          X_test  = X_test.reset_index(drop=True)[["ds"]]

          fp.fit(X_train)
          Y_pred = fp.predict(X_test)
          Y_pred = Y_pred["yhat"].copy().reset_index(drop=True)
      Y_pred = pd.Series(Y_pred)
          
      
      #rmse = mean_squared_error(Y_test,Y_pred,squared=False)
      #MAE = mean_absolute_error(Y_test,Y_pred)
      #MAPE =mean_absolute_percentage_error(Y_test,Y_pred)
      #r2 = r2_score(Y_test,Y_pred)

      #temp_df = pd.DataFrame()
      #temp_df["Truth"] = Y_test.copy().reset_index(drop=True)
      #temp_df["Predictions"] = Y_pred
      #temp_df.index = X_test.index
      #print(temp_df.head(100))
      #print("\n",temp_df.head(10))
      #print("\nModel:",model)
      #print("RMSE:",rmse)
      #print("MAE:",MAE)
      #print("MAPE:",MAPE)
      #print("R2:",r2)

      Y_test_stacked = pd.concat([Y_test_stacked,Y_test])
      Y_pred_stacked = pd.concat([Y_pred_stacked,Y_pred])



  print("\nModel:",model)
  print("RMSE:",mean_squared_error(Y_test_stacked,Y_pred_stacked,squared=False))
  print("MAE:",mean_absolute_error(Y_test_stacked,Y_pred_stacked))
  print("MAPE:",mean_absolute_percentage_error(Y_test_stacked,Y_pred_stacked))
  print("R2:",r2_score(Y_test_stacked,Y_pred_stacked))


In [ ]:
# load and preproces data
X_data, Y_data = load_train_data(weather_info=False,remove_yearly_trend=False)
model = "Random Forest"
if(model=="Linear Regression"):
  print("\n\nX data:",convert_continuous_to_discrete(X_data).head())
else:
  print("\n\nX data:",X_data.head())
# train and evaluate model
train_model(X_data,Y_data,model=model, test=True)

In [ ]:
def get_latest_predictions(X, Y):
  '''Trains every model on the full dataset(except for last week), and returns their predictions for the last week'''
  '''This is strictly for intuitive visualization purposes, not evaluation'''
  tss = TimeSeriesSplit(test_size = 2*7*24,n_splits=26)
  ind = 0

  # predictions of each model
  Y_rf = None; Y_lr= None; Y_fp = None; fp = None;
  for train_index, test_index in tss.split(X_data):
      ind+=1
      # skip all the splits except for the very last one
      if(ind!=26):
        continue

      X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

      print("\nTrain Range:",X_train.index.min(),"  ",X_train.index.max())
      print("Test Range:",X_test.index.min(),"  ",X_test.index.max())

      # Linear Regression
      X_train_temp = X_train.copy(); Y_train_temp = Y_train.copy()
      #X_train_hist, Y_train_hist = get_historical_data(60,100, X_train_temp, Y_train_temp, X_test)
      X_train_temp = convert_continuous_to_discrete(X_train_temp)
      lr = LinearRegression().fit(X_train_temp, Y_train_temp)
      X_test_temp = convert_continuous_to_discrete(X_test)
      Y_lr = lr.predict(X_test_temp)

      # Random Forest
      rf = RandomForestRegressor(random_state=0)
      rf.fit(X_train,Y_train)
      Y_rf = rf.predict(X_test)
      
      # Facebook Prophet
      fp = Prophet()
      # format data for facebook prophet
      X_train["ds"] = X_train.index.to_series().dt.tz_convert(None).copy()
      X_train = X_train.reset_index(drop=True)[["ds"]]
      X_train["y"] = Y_train.reset_index(drop=True)
      X_test["ds"] = X_test.index.to_series().dt.tz_convert(None).copy()
      X_test  = X_test.reset_index(drop=True)[["ds"]]
      # Predict
      fp.fit(X_train)
      Y_fp = fp.predict(X_test)
    
  return Y_rf, Y_lr, Y_fp, Y_test, fp

In [ ]:
def plot_predictions(Y_rf, Y_lr, Y_fp, Y_test, fp):
    '''Creates Intuitive plot each model's predictions over several days'''

    fig = fp.plot(Y_fp)
    ax = fig.gca()
    ax.set_xlim([pd.Timestamp('2022-08-28',tz="EST"), pd.Timestamp('2022-08-31',tz="EST")])
    ax.set_ylim([0, 2750])

    Y_rf = pd.Series(Y_rf)
    Y_rf.index = Y_fp["ds"]
    plt.plot(Y_rf,color='green',label="Random Forest")

    Y_lr = pd.Series(Y_lr)
    Y_lr.index =Y_fp["ds"]
    plt.plot(Y_lr,color='yellow',label="Linear Regression")

    Y_test = pd.Series(Y_test)
    Y_test.index = Y_fp["ds"]
    plt.plot(Y_test,color='black', linewidth=4,label="Observed")

    L=plt.legend()
    L.get_texts()[1].set_text('make it short')

    plt.legend(["","Facebook Prophet","Random Forest","Linear Regression","Observed"])
    plt.xlabel("Time (Day-Month-Hour)")
    plt.ylabel("Number of Trips")
    plt.title("Predictions with only Time Info",fontsize=20)
    plt.show()

In [ ]:
def plot_residuals(Y_rf, Y_lr, Y_fp, Y_test, fp):
  '''Plots the residual errors of each model'''
  plt.scatter(Y_rf,Y_test-Y_rf,s=15)
  plt.title("Random Forest Residuals")
  plt.ylim([-900,1350])
  plt.axhline(y=0.5, color='black', linestyle='-')
  plt.xlabel("Y Predicted")
  plt.ylabel("Y Truth - Y Predicted")
  plt.show()

  plt.scatter(Y_lr,Y_test-Y_lr,s=15)
  plt.title("Linear Regression Residuals")
  plt.ylim([-900,1350])
  plt.axhline(y=0.5, color='black', linestyle='-')
  plt.xlabel("Y Predicted")
  plt.ylabel("Y Truth - Y Predicted")
  plt.show()

  Y_fp = Y_fp["yhat"].to_numpy()
  print(Y_fp.size)
  print(Y_test.size)
  plt.scatter(Y_fp,Y_test-Y_fp,s=15)
  plt.title("Facebook Prophet Residuals")
  plt.ylim([-900,1350])
  plt.axhline(y=0.5, color='black', linestyle='-')
  plt.xlabel("Y Predicted")
  plt.ylabel("Y Truth - Y Predicted")
  plt.show()


In [ ]:
# create plots for each models
X_data, Y_data = load_train_data(weather_info=False,remove_yearly_trend=False)
Y_rf, Y_lr, Y_fp, Y_test, fp = get_latest_predictions(X_data, Y_data)
plot_predictions(Y_rf, Y_lr, Y_fp, Y_test, fp)
plot_residuals(Y_rf, Y_lr, Y_fp, Y_test, fp)